# VGGNet

In [2]:
import tensorflow as tf
import numpy as tf
import DL_utils as dl

In [3]:
def create_placeholders(n_H, n_W, n_C, n_y):
    '''
    Function to create placeholder for tensorflow session
    
    Args:
    n_H = height of the image
    n_W = width of image
    n_C = number of channels
    n_y = number of output features
    
    returns:
    X,Y
    '''
    X = tf.placeholder(tf.float32, shape = (None, n_H, n_W, n_C))
    Y = tf.placeholder(tf.float32, shape = (None, n_y))
    
    return X, Y

In [4]:
def VGG_forward_propagation(X, parameters):
    '''
    Forward propagation for AlexNet without normalization
    
    '''
    # 1st conv
    A1 = dl.conv_layer(X,parameters['W1'],parameters['b1'], stride=[1,1,1,1], padding='SAME', name='1-1')
    # 2nd conv
    A2 = dl.conv_layer(X,parameters['W2'],parameters['b2'], stride=[1,1,1,1], padding='SAME', name='1-2')
    # 1st pool
    P1 = dl.max_pool(A2, kernel=[1,2,2,1], strides=[1,2,2,1], padding = 'VALID', name='1')
    # 3rd conv
    A3 = dl.conv_layer(P1,parameters['W3'],parameters['b3'], strides=[1,1,1,1], padding='SAME', name='2-1')
    # 4th conv
    A4 = dl.conv_layer(A3,parameters['W4'],parameters['b4'], strides=[1,1,1,1], padding='SAME', name='2-2')
    #2nd pool
    P2 = dl.max_pool(A4, kernel=[1,2,2,1], strides=[1,2,2,1], padding = 'VALID', name='2')
    # 5th conv
    A5 = dl.conv_layer(P2,parameters['W5'], parameters['b5'], strides=[1,1,1,1], padding='SAME', name='3-1')
    # 6th conv
    A6 = dl.conv_layer(A5,parameters['W6'], parameters['b6'], strides=[1,1,1,1], padding='SAME', name='3-2')
    # 3rd pool
    P3 = dl.max_pool(A6, kernel=[1,2,2,1], strides=[1,2,2,1], padding = 'VALID', name='3')
    # 7th conv
    A7 = dl.conv_layer(P3,parameters['W7'], parameters['b7'], strides=[1,1,1,1], padding='SAME', name='4-1')
    # 8th conv
    A8 = dl.conv_layer(A7,parameters['W8'], parameters['b8'], strides=[1,1,1,1], padding='SAME', name='4-2')
    # 9th conv
    A9 = dl.conv_layer(A8,parameters['W9'], parameters['b9'], strides=[1,1,1,1], padding='SAME', name='4-3')
    # 4th pool
    P4 = dl.max_pool(A9, kernel=[1,2,2,1], strides=[1,2,2,1], padding = 'VALID', name='4')
    # 10th conv
    A10 = dl.conv_layer(P4,parameters['W10'], parameters['b10'], strides=[1,1,1,1], padding='SAME', name='5-1')
    # 11th conv
    A11 = dl.conv_layer(A10,parameters['W11'], parameters['b11'], strides=[1,1,1,1], padding='SAME', name='5-2')
    # 12th conv
    A12 = dl.conv_layer(A11,parameters['W12'], parameters['b12'], strides=[1,1,1,1], padding='SAME', name='5-3')
    # 5th pool
    P5 = dl.max_pool(A12, kernel=[1,2,2,1], strides=[1,2,2,1], padding = 'VALID', name='5')
    # Flattening the last pooling layer
    P5 = tf.contrib.layers.flatten(P5)
    # FC1 - 4096 neurons
    F1 = dl.fc_layer(P5, 4096, activation_fn=None, name='1')
    # FC2 - 4096 neurons
    F2 = dl.fc_layer(F1, 4096, activation_fn=None, name='2')
    # FC3 - 1000 neurons reping different classes - may be modified for other models
    F3 = dl.fc_layer(F2, 1000, activation_fn=None, name='3')
    
    return F3

In [5]:
def compute_AN_cost(Z, Y):
    '''
    Computes cost 
    
    Args:
    Z8 -- Logits/Linear output from the last fully connected layer
    Y -- labels corresponding to each example in the batch
    
    Returns:
    cost -- cost of the epoch
    '''
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z, labels = Y))
    
    return cost

In [ ]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.009,
          num_epochs = 100, minibatch_size = 64, print_cost = True):
    '''
    Implements the AlexNet model for image classification
    
    Args:
    X_train -- training set, of shape (None, 64, 64, 3)
    Y_train -- test set, of shape (None, n_y = 6)
    X_test -- training set, of shape (None, 64, 64, 3)
    Y_test -- test set, of shape (None, n_y = 6)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    returns:
    train_accuracy -- real number, accuracy on the train set (X_train)
    test_accuracy -- real number, testing accuracy on the test set (X_test)
    parameters -- parameters learnt by the model. They can then be used to predict.
    '''
    # retrieve necessary tensor dimensions
    (m, n_H0, n_W0, n_C0) = X_train.shape
    n_y = Y_train.shape[1]
    seed = 3
    
    # Create placeholders for the input and output tensors
    X, Y = create_placeholders(n_H0, n_W0, n_C0, n_y)
    
    # create dictionary to assign tensor shapes to the weight filters for AlexNet
    parameter_size_dict = {"W1":[11,11,3,96], "W2":[5,5,96,256], "W3":[3,3,256,384], "W4":[3,3,384,384], "W5":[3,3,384,256]}
    
    # Initialize parameters
    parameters = initialize_parameters(parameter_size_dict)
    
    # apply forward propogation
    Z = forward_propogation(X, parameters)
    
    # compute cost 
    cost = compute_cost(Z, Y)
    
    # define an optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    # initialize global variables
    init = tf.global_variables_initializer()
    
    # run the session and train
    with tf.Session() as sess:
        
        # run the session
        sess.run(init)
        
        # training loop
        for epoch in num_epochs:
            
            # split into minibatches
            minibatch_cost = 0.
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
            
            for minibatch in minibatches:
                
                # select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                # computation of the actual graph
                _, temp_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
                
                # add the cost
                minibatch_cost += temp_cost / num_minibatches
                
            # Print the cost every epoch
            if print_cost == True and epoch % 5 == 0:
                print ("Cost after epoch %i: %f" % (epoch, minibatch_cost))
            if print_cost == True and epoch % 1 == 0:
                costs.append(minibatch_cost)
                
                
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()

    # Calculate the correct predictions
    predict_op = tf.argmax(Z3, 1)
    correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))

    # Calculate accuracy on the test set
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print(accuracy)
    train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
    test_accuracy = accuracy.eval({X: X_test, Y: Y_test})
    print("Train Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)

    return train_accuracy, test_accuracy, parameters